In [ ]:
!curl -o "aihubshell" https://api.aihub.or.kr/api/aihubshell.do
!chmod +x aihubshell
AIHUB_ID = "ryugibo@daum.net" # @param {type:"string"}
AIHUB_PW = "" # @param {type:"string"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6634  100  6634    0     0  39779      0 --:--:-- --:--:-- --:--:-- 39963


In [ ]:
%cd CVProject

/content/CVProject


In [ ]:
!../aihubshell -mode l -datasetkey 562 -filekey 51881

aihubshell version 24.01.29 v0.3
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─153.반려동물 안구질환 데이터
        ├─02.저작도구
        │  ├─저작도구 소스코드.zip | 153 MB | 55226
        │  └─저작도구 설명서.zip | 4 MB | 55227
        └─01.데이터
            ├─1.Training
            │  ├─라벨링데이터
            │  │  ├─TL1.zip | 5 GB | 51880
            │  │  └─TL2.zip | 7 GB | 51881
            │  └─원천데이터
            │      ├─TS1.zip | 76 GB | 51882
            │      └─TS2.zip | 94 GB | 51883
            └─2.Validation
                ├─라벨링데이터
                │  └─VL.zip | 1 GB | 51884
                └─원천데이터
                    └─VS.zip | 18 GB | 51885


In [ ]:
!../aihubshell -mode d -datasetkey 562 -filekey 51880,51881,51884 -aihubpw {AIHUB_PW} -aihubid {AIHUB_ID}

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12.9G    0 12.9G    0     0  18.3M      0 --:--:--  0:12:00 --:--:-- 18.8M
Request successful with HTTP status 200.
Download successful.
153.반려동물_안구질환_데이터/01.데이터/2.Validation/라벨링데이터/VL.zip.part0
153.반려동물_안구질환_데이터/01.데이터/2.Validation/라벨링데이터/VL.zip.part1073741824
153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip.part0
153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip.part1073741824
153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip.part2147483648
153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip.part3221225472
153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip.part4294967296
153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip.part5368709120
153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip

In [ ]:
!unzip -l /content/CVProject/153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL1.zip -x */*/안구초음파**

In [ ]:
!unzip -q /content/CVProject/153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL1.zip -d ./raw_datas -x */*/안구초음파**

In [ ]:
!unzip -q /content/CVProject/153.반려동물_안구질환_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip -d ./raw_datas -x */*/안구초음파**

In [ ]:
!unzip -q /content/CVProject/153.반려동물_안구질환_데이터/01.데이터/2.Validation/라벨링데이터/VL.zip -d ./raw_datas -x */*/안구초음파**

In [ ]:
import os
os.getcwd()

'/content'

In [2]:
import glob
import json
import os
from PIL import Image, ImageChops
from tqdm import tqdm


def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)
    return im


for desease in os.listdir("CVProject/raw_datas/개/안구/일반"):
    if desease.startswith("."):
        continue
    desease_path = f"CVProject/raw_datas/개/안구/일반/{desease}"
    for sick in os.listdir(desease_path):
        if sick.startswith("."):
            continue
        sick_path = f"{desease_path}/{sick}"
        print(f"{desease}({sick})", end=" >> ")
        for meta_path in tqdm(glob.glob(f"{sick_path}/*.json"), position=0, leave=True):
            f = open(meta_path, "r")
            meta = json.load(f)
            f.close()

            img_filename = meta['label']['label_filename']
            img_filepath = f"{os.path.dirname(meta_path)}/{img_filename}"

            img = None
            if os.path.isfile(img_filepath):
                img = Image.open(img_filepath)
            elif os.path.isfile(f"{img_filepath}.jpg"):
                img = Image.open(f"{img_filepath}.jpg")
            elif os.path.isfile(f"{img_filepath}.jpeg"):
                img = Image.open(f"{img_filepath}.jpeg")
            elif os.path.isfile(f"{img_filepath}.png"):
                img = Image.open(f"{img_filepath}.png")

            if img is None:
                print(f"'{img_filepath}' is not exist. in '{meta_path}'")
                continue

            if "." not in img_filepath:
                print(f"NO EXTENSION {desease}({sick}) {img_filepath}")
                img_filepath = f"{img_filepath}.jpg"

            img = trim(img)

            min_size = min(img.width, img.height)
            x_crop = img.width - min_size
            y_crop = img.height - min_size
            img = img.crop((x_crop / 2, y_crop / 2, img.width - (x_crop / 2), img.height - (y_crop / 2)))
            img = img.resize((224, 224), Image.Resampling.LANCZOS)

            save_dir = f"CVProject/개_안구_resized_keep/{desease}/{sick}"
            os.makedirs(save_dir, exist_ok=True)
            img.save(f"{save_dir}/{img_filename.replace('.png', '.jpg')}", "JPEG")

결막염(무) >> 

100%|██████████| 10800/10800 [11:29<00:00, 15.67it/s]


결막염(유) >> 

100%|██████████| 10801/10801 [09:02<00:00, 19.93it/s]


궤양성각막질환(무) >> 

...

100%|██████████| 10797/10797 [09:41<00:00, 18.58it/s]


핵경화(무) >> 

100%|██████████| 10800/10800 [09:14<00:00, 19.47it/s]


핵경화(유) >> 

100%|██████████| 10798/10798 [07:56<00:00, 22.65it/s]


In [3]:
basepath = "CVProject/개_안구_resized_keep"
for desease in os.listdir(basepath):
    if desease.startswith("."):
        continue
    desease_path = f"{basepath}/{desease}"
    for sick in os.listdir(desease_path):
        if sick.startswith("."):
            continue
        sick_path = f"{desease_path}/{sick}"
        count = !ls {sick_path} | wc -l
        print(f"{desease}/{sick} : {count}")

결막염/무 : ['10800']
결막염/유 : ['10801']
궤양성각막질환/무 : ['7740']
궤양성각막질환/상 : ['7727']
궤양성각막질환/하 : ['7741']
백내장/무 : ['7740']
백내장/비성숙 : ['7758']
백내장/성숙 : ['7727']
백내장/초기 : ['7738']
비궤양성각막질환/무 : ['5400']
비궤양성각막질환/상 : ['5399']
비궤양성각막질환/하 : ['5404']
색소침착성각막염/무 : ['7920']
색소침착성각막염/유 : ['7922']
안검내반증/무 : ['10799']
안검내반증/유 : ['10799']
안검염/무 : ['7740']
안검염/유 : ['7738']
안검종양/무 : ['5400']
안검종양/유 : ['5386']
유루증/무 : ['10800']
유루증/유 : ['10797']
핵경화/무 : ['10800']
핵경화/유 : ['10798']


In [7]:
import os
import shutil
from tqdm import tqdm

basepath = "CVProject/개_안구_resized_keep"
targetpath = "CVProject/개_안구_resized_train_test"
for desease in os.listdir(basepath):
    if desease.startswith("."):
        continue
    desease_path = f"{basepath}/{desease}"
    for sick in os.listdir(desease_path):
        if sick.startswith("."):
            continue
        sick_path = f"{desease_path}/{sick}"
        images = os.listdir(sick_path)

        target_test_path = f"{targetpath}/{desease}_test/{sick}"
        os.makedirs(target_test_path, exist_ok=True)
        print(target_test_path, end=" >> ")
        for test_image in tqdm(images[:100], position=0, leave=True):
            shutil.copyfile(f"{sick_path}/{test_image}", f"{target_test_path}/{test_image}")

        target_train_path = f"{targetpath}/{desease}_train/{sick}"
        os.makedirs(target_train_path, exist_ok=True)
        print(target_train_path, end=" >> ")
        for train_image in tqdm(images[100:], position=0, leave=True):
            shutil.copyfile(f"{sick_path}/{train_image}", f"{target_train_path}/{train_image}")

CVProject/개_안구_resized_train_test/결막염_test/무 >> 

100%|██████████| 100/100 [00:01<00:00, 63.17it/s]


CVProject/개_안구_resized_train_test/결막염_train/무 >> 

100%|██████████| 10700/10700 [03:19<00:00, 53.73it/s]


CVProject/개_안구_resized_train_test/결막염_test/유 >> 

...

100%|██████████| 10700/10700 [03:28<00:00, 51.21it/s]


CVProject/개_안구_resized_train_test/핵경화_test/유 >> 

100%|██████████| 100/100 [00:02<00:00, 47.99it/s]


CVProject/개_안구_resized_train_test/핵경화_train/유 >> 

100%|██████████| 10698/10698 [03:26<00:00, 51.91it/s]


In [8]:
basepath = "CVProject/개_안구_resized_train_test"
for desease in os.listdir(basepath):
    if desease.startswith("."):
        continue
    desease_path = f"{basepath}/{desease}"
    for sick in os.listdir(desease_path):
        if sick.startswith("."):
            continue
        sick_path = f"{desease_path}/{sick}"
        count = !ls {sick_path} | wc -l
        print(f"{desease}/{sick} : {count}")

결막염_test/무 : ['100']
결막염_test/유 : ['100']
결막염_train/무 : ['10700']
결막염_train/유 : ['10701']
궤양성각막질환_test/무 : ['100']
궤양성각막질환_test/상 : ['100']
궤양성각막질환_test/하 : ['100']
궤양성각막질환_train/무 : ['7640']
궤양성각막질환_train/상 : ['7627']
궤양성각막질환_train/하 : ['7641']
백내장_test/무 : ['100']
백내장_test/비성숙 : ['100']
백내장_test/성숙 : ['100']
백내장_test/초기 : ['100']
백내장_train/무 : ['7640']
백내장_train/비성숙 : ['7658']
백내장_train/성숙 : ['7627']
백내장_train/초기 : ['7638']
비궤양성각막질환_test/무 : ['100']
비궤양성각막질환_test/상 : ['100']
비궤양성각막질환_test/하 : ['100']
비궤양성각막질환_train/무 : ['5300']
비궤양성각막질환_train/상 : ['5299']
비궤양성각막질환_train/하 : ['5304']
색소침착성각막염_test/무 : ['100']
색소침착성각막염_test/유 : ['100']
색소침착성각막염_train/무 : ['7820']
색소침착성각막염_train/유 : ['7822']
안검내반증_test/무 : ['100']
안검내반증_test/유 : ['100']
안검내반증_train/무 : ['10699']
안검내반증_train/유 : ['10699']
안검염_test/무 : ['100']
안검염_test/유 : ['100']
안검염_train/무 : ['7640']
안검염_train/유 : ['7638']
안검종양_test/무 : ['100']
안검종양_test/유 : ['100']
안검종양_train/무 : ['5300']
안검종양_train/유 : ['5286']
유루증_test/무 : ['100']
유

In [ ]:
!zip -qr CVProject/개_안구_squared_all.zip CVProject/개_안구_squared_all